In [4]:
# ==========================================
# AI Examiner – Decision Quality Evaluation
# ==========================================

import matplotlib.pyplot as plt
import string


# ---------------- LOGIC SCORE ----------------
def logic_score(reasoning):
    positive_keywords = [
        "because", "therefore", "as a result",
        "this will lead to", "hence"
    ]

    negative_keywords = [
        "but", "however", "although"
    ]

    reasoning_lower = reasoning.lower()
    score = 0

    for word in positive_keywords:
        if word in reasoning_lower:
            score += 8

    for word in negative_keywords:
        if word in reasoning_lower:
            score -= 5

    return max(0, min(score, 25))


# ---------------- RISK SCORE ----------------
def risk_score(reasoning):
    risk_keywords = [
        "risk", "drawback", "downside", "challenge",
        "may fail", "could lead to", "possible loss",
        "uncertainty", "limitation"
    ]

    reasoning_lower = reasoning.lower()
    score = 0

    for word in risk_keywords:
        if word in reasoning_lower:
            score += 5

    return min(score, 20)


# ---------------- BIAS SCORE ----------------
def bias_score(reasoning):
    absolute_words = [
        "always", "never", "everyone", "no one", "guaranteed"
    ]

    emotional_words = [
        "obviously", "clearly", "definitely",
        "best", "worst", "useless"
    ]

    reasoning_lower = reasoning.lower()
    score = 15  # full marks

    for word in absolute_words:
        if word in reasoning_lower:
            score -= 3

    for word in emotional_words:
        if word in reasoning_lower:
            score -= 2

    return max(score, 0)


# ---------------- COMPLETENESS SCORE ----------------
def completeness_score(scenario, reasoning):
    scenario_clean = scenario.lower().translate(
        str.maketrans("", "", string.punctuation)
    )
    reasoning_clean = reasoning.lower().translate(
        str.maketrans("", "", string.punctuation)
    )

    scenario_words = set(scenario_clean.split())
    reasoning_words = set(reasoning_clean.split())

    stopwords = {
        "the", "is", "are", "a", "an", "and", "to", "of",
        "in", "on", "for", "with", "as", "by", "this",
        "that", "will", "be"
    }

    scenario_words -= stopwords
    reasoning_words -= stopwords

    if not scenario_words:
        return 0

    common_words = scenario_words.intersection(reasoning_words)
    coverage_ratio = len(common_words) / len(scenario_words)

    return min(int(coverage_ratio * 20), 20)


# ---------------- ETHICS SCORE ----------------
def ethics_score(reasoning):
    ethical_red_flags = [
        "only", "exclude", "reject", "deny",
        "prioritize younger", "fire", "terminate",
        "remove support", "no impact",
        "does not matter", "irrelevant"
    ]

    reasoning_lower = reasoning.lower()
    score = 20  # full marks

    for word in ethical_red_flags:
        if word in reasoning_lower:
            score -= 4

    return max(score, 0)


# ---------------- SCORE DECISION ----------------
def score_decision(decision_data):
    return {
        "logic": logic_score(decision_data["reasoning"]),
        "risk": risk_score(decision_data["reasoning"]),
        "bias": bias_score(decision_data["reasoning"]),
        "completeness": completeness_score(
            decision_data["scenario"],
            decision_data["reasoning"]
        ),
        "ethics": ethics_score(decision_data["reasoning"])
    }


# ---------------- EXPLANATION ----------------
def generate_explanation(scores):
    explanations = []

    explanations.append(
        "Logic: Reasoning lacks clear cause–effect explanation."
        if scores["logic"] < 10
        else "Logic: Reasoning shows acceptable cause–effect clarity."
    )

    explanations.append(
        "Risk: Decision does not acknowledge possible downsides or uncertainty."
        if scores["risk"] < 5
        else "Risk: Decision considers potential risks."
    )

    explanations.append(
        "Bias: Reasoning contains emotionally loaded or absolute language."
        if scores["bias"] < 10
        else "Bias: Reasoning is objective and neutral."
    )

    explanations.append(
        "Completeness: Reasoning does not fully address key aspects of the scenario."
        if scores["completeness"] < 8
        else "Completeness: Reasoning sufficiently addresses the scenario."
    )

    explanations.append(
        "Ethics: Decision may unfairly impact certain stakeholders."
        if scores["ethics"] < 10
        else "Ethics: No major ethical concerns detected."
    )

    return explanations


# ---------------- VERDICT ----------------
def decision_verdict(total_score):
    if total_score >= 60:
        return "Good Decision", "Reasoning is strong, balanced, and well-considered."
    elif total_score >= 40:
        return "Risky Decision", "Decision has merit but shows notable weaknesses."
    else:
        return "Poor Decision", "Decision lacks sufficient reasoning quality and balance."


# ---------------- VISUALIZATION ----------------
def visualize_scores(scores):
    dimensions = list(scores.keys())
    values = list(scores.values())

    plt.figure()
    plt.bar(dimensions, values)
    plt.ylim(0, 25)
    plt.xlabel("Evaluation Dimensions")
    plt.ylabel("Score")
    plt.title("Decision Quality Breakdown")
    plt.show()


# ---------------- MAIN ----------------
def main():
    print("=== AI Examiner: Decision Quality Evaluation ===\n")

    decision_data = {
        "scenario": input("Enter scenario: "),
        "decision": input("Enter decision taken: "),
        "reasoning": input("Enter reasoning: ")
    }

    scores = score_decision(decision_data)

    print("\n--- Score Breakdown ---")
    for key, value in scores.items():
        print(f"{key.capitalize()} Score: {value}")

    total_score = sum(scores.values())
    print("\nTotal Score (out of 100):", total_score)

    explanations = generate_explanation(scores)

    print("\n--- Explanation ---")
    for exp in explanations:
        print("-", exp)

    verdict, verdict_reason = decision_verdict(total_score)

    print("\n--- Final Verdict ---")
    print("Verdict:", verdict)
    print("Summary:", verdict_reason)

    visualize_scores(scores)


if __name__ == "__main__":
    main()


=== AI Examiner: Decision Quality Evaluation ===



Enter scenario:  A company is experiencing declining profits and employee dissatisfaction.
Enter decision taken:  Lay off employees immediately.
Enter reasoning:  Laying off employees will reduce costs. This decision is obviously correct and will always improve profits. Employee concerns do not matter.



--- Score Breakdown ---
Logic Score: 0
Risk Score: 0
Bias Score: 10
Completeness Score: 6
Ethics Score: 20

Total Score (out of 100): 36

--- Explanation ---
- Logic: Reasoning lacks clear cause–effect explanation.
- Risk: Decision does not acknowledge possible downsides or uncertainty.
- Bias: Reasoning is objective and neutral.
- Completeness: Reasoning does not fully address key aspects of the scenario.
- Ethics: No major ethical concerns detected.

--- Final Verdict ---
Verdict: Poor Decision
Summary: Decision lacks sufficient reasoning quality and balance.


C:\Users\Admin\AppData\Local\Temp\ipykernel_14352\1969629861.py:198: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
